# Hybrid CNN-RNN Model using TensorFlow

### Importing Clean Data & Dependencies

In [1]:
from data_clean import df_prediction as df
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

### Preparing Data for Model Training

Reformatting Date Column

In [2]:
df = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.loc[:, 'Day'] = df['Date'].dt.day
df.loc[:, 'Month'] = df['Date'].dt.month
df.loc[:, 'Year'] = df['Date'].dt.year
df = df.drop(columns=['Date'])
df


,Latitude,Longitude,Depth,Magnitude,Day,Month,Year
0,-9.18,119.06,10,4.9,1,11,2008
1,-6.55,129.64,10,4.6,1,11,2008
2,-7.01,106.63,121,3.7,1,11,2008
3,-3.30,127.85,10,3.2,1,11,2008
4,-6.41,129.54,70,4.3,1,11,2008
...,...,...,...,...,...,...,...
81218,-8.02,121.81,69,3.4,10,1,2022
81219,1.50,127.85,10,3.7,10,1,2022
81220,1.50,127.86,10,3.0,10,1,2022
81221,1.48,127.86,10,2.7,10,1,2022


Define features and target columns

In [3]:
feature_columns = ['Latitude', 'Longitude', 'Depth', 'Day', 'Month', 'Year']
target_columns = ['Magnitude', 'Latitude', 'Longitude', 'Day', 'Month', 'Year']

In [4]:
X = df[feature_columns]
y = df[target_columns]

Normalizing X

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Splitting the data

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Building the Model

In [7]:
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)

Creating one output layer for each target

In [8]:
magnitude_output = Dense(1, name='magnitude')(x)
latitude_output = Dense(1, name='latitude')(x)
longitude_output = Dense(1, name='longitude')(x)
day_output = Dense(1, name='day')(x) # For day prediction
month_output = Dense(1, name='month')(x) # For month prediction
year_output = Dense(1, name='year')(x) # For year prediction

In [9]:
model = Model(inputs=input_layer, outputs=[magnitude_output, latitude_output, longitude_output, day_output, month_output, year_output])

Compiling the model

In [10]:
model.compile(optimizer=Adam(), loss='mse')

### Training the Model

In [11]:
model.fit(X_train, [y_train['Magnitude'], y_train['Latitude'], y_train['Longitude'], y_train['Day'], y_train['Month'], y_train['Year']], 
          epochs=500, batch_size=32, validation_split=0.2)

Epoch 1/500


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 2s 568us/step - loss: 2367056.5000 - val_loss: 3537.2151
Epoch 2/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 539us/step - loss: 113786.0078 - val_loss: 2435.4202
Epoch 3/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 538us/step - loss: 91489.4297 - val_loss: 3433.5469
Epoch 4/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 522us/step - loss: 74117.9531 - val_loss: 2399.7695
Epoch 5/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 618us/step - loss: 61822.1797 - val_loss: 4480.8218
Epoch 6/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 575us/step - loss: 52457.3398 - val_loss: 3753.8562
Epoch 7/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 527us/step - loss: 42765.2305 - val_loss: 2149.8694
Epoch 8/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - loss: 34674.0156 - val_loss: 1857.7297
Epoch 9/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - loss: 25709.1055 - val_loss: 1317.6285
Epoch 10/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 514us/step - loss: 17201.2422 - val_loss: 1041.8903
Epoch 11/500
1625/1625 ━━━━━

### Results

In [12]:
predictions = model.predict(X_test)

magnitude_predictions = predictions[0]
latitude_predictions = predictions[1]
longitude_predictions = predictions[2]
day_predictions = predictions[3]
month_predictions = predictions[4]
year_predictions = predictions[5]


508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step


In [13]:
print(f"Magnitude = {magnitude_predictions[0][0]}")
print(f"Latitude = {latitude_predictions[0][0]}")
print(f"Longitude = {longitude_predictions[0][0]}")
print(f"Day = {day_predictions[0][0]}")
print(f"Month = {month_predictions[0][0]}")
print(f"Year = {year_predictions[0][0]}")

Magnitude = 4.2132439613342285
Latitude = -1.7106192111968994
Longitude = 131.90911865234375
Day = 7.823936939239502
Month = 2.751948833465576
Year = 2010.5975341796875


### Evaluating the Model

In [14]:
model.evaluate(X_test, [y_test['Magnitude'], y_test['Latitude'], y_test['Longitude']])

508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 209us/step - loss: 2.0923


2.106126546859741